In [5]:
import numpy as np

In [6]:
from dipy.io.stateful_tractogram import StatefulTractogram, Space
from tqdm import tqdm

class MatlabFileTracking():
    def __init__(self, mat_file, fg_ref):
        self.mat_file = mat_file
        self.fg_ref = fg_ref
    def __iter__(self):
        for i in tqdm(range(self.fg_ref.shape[1])):
            yield self.mat_file[self.fg_ref[0, i]][:]

def track_matlab_file(mat_file, reference):
    fg_ref = mat_file['fg']['fibers'][:]
    tracker = MatlabFileTracking(mat_file, fg_ref)
    return StatefulTractogram(tracker, reference, Space.RASMM)
# could be T1 weighted or diffusion, try each then open up b0 volume
# not in volume coordinates (lets compare to get_streamlines)
# origin might not matter, try one of center / corner and see if they shift (prolly by half a voxel)

In [7]:
import os.path as op
import os

import h5py
import s3fs

from dipy.io.streamline import save_tractogram
import nibabel as nib
# from scipy.io import loadmat

fs = s3fs.S3FileSystem()
local_dir = 'm2p_tracts'
if not op.exists(local_dir):
    os.mkdir(local_dir)

subjects = [
    1110,
    1112,
    1113,
    1103,
    1184,
    1208,
    1210,
    1241,
    1262,
    1271]
for subject in subjects:
    # Download necessary files
    dwi_dir = f'prek-diffusion/derivatives/sub-{subject}/dtiInit_ses-pre'
    fs.get(op.join(dwi_dir, f"dti32_allvols_rt/fibers/WholeBrainFG.mat"),
           op.join(local_dir, f"WholeBrainFG-{subject}.mat"))
#     fs.get(op.join(dwi_dir, f"sub-{subject}_ses-pre_acq-b1500_dwi_aligned_trilin_acpcXform.mat"),
#            op.join(local_dir, f"sub-{subject}_acpcXform.mat"))
    fs.get(op.join(dwi_dir, f"sub-{subject}_ses-pre_acq-b1500_dwi_aligned_trilin.nii.gz"),
           op.join(local_dir, f"sub-{subject}_dwi.nii.gz"))

    # Open Files
    mat_file = h5py.File(op.join(local_dir, f"WholeBrainFG-{subject}.mat"), 'r')
    #xform_file = loadmat(op.join(local_dir, f"sub-{subject}_acpcXform.mat"))
    img = nib.load(op.join(local_dir, f"sub-{subject}_dwi.nii.gz"))

    # track from matlab file and save
    #affine = xform_file['acpcXform']
    sft = track_matlab_file(mat_file, img)
    mat_file.close()
    save_tractogram(sft, f"m2p_tracts/tract_test_sub-{subject}.trk", bbox_valid_check=False)


100%|██████████| 104695/104695 [00:25<00:00, 4112.02it/s]
